# Часть 2

## Импорт библиотек

In [2]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

import pickle
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV

## Modeling

### Подготовка наборов

In [3]:
df_prepared = pd.read_csv('data/df_prepared.csv', low_memory=False)

In [4]:
df_prepared.shape

(1860042, 408)

In [5]:
df_prepared.head()

,target,utm_source_BHcvLfOaCWvWTykYqHVe,utm_source_GpAkIXsclxDGyILfNlrR,utm_source_ISrKoXQCxqqYvAZICvjs,utm_source_IZEXUFLARCUMynmHNBGo,utm_source_MvfHsxITijuriZxsqZqt,utm_source_PlbkrSYoHuZBWfYjYnfw,utm_source_QxAxdyPLuQMEcrdZWdWb,utm_source_RmEBuqrriAfAVsLQQmhk,utm_source_SzZERoLMmrEUEhDaYcyN,...,utm_medium_type_organic,utm_medium_type_other,timeofday_day,timeofday_evening,timeofday_morning,timeofday_night,visit_number_std,device_screen_area_std,month_std,dayofweek_std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,-0.553399,-0.498629,0.760687,-0.403088
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.553399,-0.399837,0.760687,1.613486
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,-0.553399,-0.498629,1.215280,-0.907232
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,-0.553399,-0.428077,-1.966869,1.109343
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.853329,-0.428077,-1.966869,1.109343


In [6]:
print(f'Размер несбалансированного датасета: {df_prepared.shape}')
print(f'Количество объектов с ЦД = 0: {len(df_prepared[df_prepared.target == 0])}')
print(f'Количество объектов с ЦД = 1: {len(df_prepared[df_prepared.target == 1])}')

Размер несбалансированного датасета: (1860042, 408)
Количество объектов с ЦД = 0: 1809728
Количество объектов с ЦД = 1: 50314


Сейчас наши данные сильно несбалансированные по ключевому признаку. Процент объектов с выполненным ЦД составляет менее 3 процентов от общего числа объектов. Такой перекос будет искажать результаты моделирования. Проверим модель машинного обучения методом случайного леса по исходным и сбалансированным данным.

#### Несбалансированный набор входных данных

In [7]:
x = df_prepared.drop(['target'], axis=1)

In [8]:
y = df_prepared['target']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [10]:
x_train.shape, x_test.shape

((1302029, 407), (558013, 407))

#### Сбалансированный набор входных данных

In [11]:
df_prepared_1 = df_prepared[df_prepared.target == 1]
df_prepared_0 = df_prepared[df_prepared.target == 0]
df_prepared_0 = df_prepared_0.iloc[: 50314]
df_prepared_balanced = pd.concat([df_prepared_1, df_prepared_0], axis=0)
df_prepared_balanced = df_prepared_balanced.sample(frac=1).reset_index(drop=True)

In [12]:
print(f'Размер сбалансированного датасета: {df_prepared_balanced.shape}')
print(f'Количество объектов с ЦД = 0: {len(df_prepared_balanced[df_prepared_balanced.target == 0])}')
print(f'Количество объектов с ЦД = 1: {len(df_prepared_balanced[df_prepared_balanced.target == 1])}')

Размер сбалансированного датасета: (100628, 408)
Количество объектов с ЦД = 0: 50314
Количество объектов с ЦД = 1: 50314


In [13]:
x_bal = df_prepared_balanced.drop(['target'], axis=1)
y_bal = df_prepared_balanced['target']

In [14]:
x_train_bal, x_test_bal, y_train_bal, y_test_bal = train_test_split(x_bal, y_bal, test_size=0.3, random_state=42)

In [15]:
x_train_bal.shape, x_test_bal.shape

((70439, 407), (30189, 407))

### Случайный лес

#### По несбалансированным данным

In [9]:
rf_clf = RandomForestClassifier()
rf_clf.fit(x_train, y_train)

RandomForestClassifier()

In [10]:
predicted_train_rf = rf_clf.predict(x_train)
predicted_test_rf = rf_clf.predict(x_test)

In [14]:
rf_clf.predict_proba(x_test)

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [0.94880415, 0.05119585],
       ...,
       [0.99      , 0.01      ],
       [0.8125    , 0.1875    ],
       [0.97600397, 0.02399603]])

In [29]:
print('Результаты моделирования случайного леса с базовыми параметрами по несбалансированным данным:')
print('ROC AUC точность модели:', roc_auc_score(y_test, rf_clf.predict_proba(x_test)[:, 1]))
print(f'Точность на тестовой выборке: {accuracy_score(y_test, predicted_test_rf)}')
print(f'Матрица ошибок:\n {confusion_matrix(y_test, predicted_test_rf)}')

Результаты моделирования случайного леса с базовыми параметрами по несбалансированным данным:
ROC AUC точность модели: 0.619109429909404
Точность на тестовой выборке: 0.9689236630687815
Матрица ошибок:
 [[540402   2502]
 [ 14839    270]]


#### По сбалансированным данным

In [16]:
rf_clf_bal = RandomForestClassifier()
rf_clf_bal.fit(x_train_bal, y_train_bal)

RandomForestClassifier()

In [17]:
predicted_train_rf_bal = rf_clf_bal.predict(x_train_bal)
predicted_test_rf_bal = rf_clf_bal.predict(x_test_bal)

In [18]:
print('Результаты моделирования случайного леса с базовыми параметрами по сбалансированным данным:')
print('ROC AUC точность модели:', roc_auc_score(y_test_bal, rf_clf_bal.predict_proba(x_test_bal)[:, 1]))
print(f'Точность на тестовой выборке: {accuracy_score(y_test_bal, predicted_test_rf_bal)}')
print(f'Матрица ошибок:\n {confusion_matrix(y_test_bal, predicted_test_rf_bal)}')

Результаты моделирования случайного леса с базовыми параметрами по сбалансированным данным:
ROC AUC точность модели: 0.6916253779089143
Точность на тестовой выборке: 0.6380800953989864
Матрица ошибок:
 [[9590 5556]
 [5370 9673]]


#### Оценка влияния сбалансированности входных данных

In [32]:
balance_rf_results = {'Тип входных данных': ['Несблалансированные', 'Сбалансированные'], 
           'ROC AUC': [roc_auc_score(y_test, rf_clf.predict_proba(x_test)[:, 1]), roc_auc_score(y_test_bal, rf_clf_bal.predict_proba(x_test_bal)[:, 1])],
           'Точность на тестовой выборке': [accuracy_score(y_test, predicted_test_rf), accuracy_score(y_test_bal, predicted_test_rf_bal)]}

In [34]:
df_balance_rf_results = pd.DataFrame(data=balance_rf_results)

In [35]:
df_balance_rf_results

,Тип входных данных,ROC AUC,Точность на тестовой выборке
0,Несблалансированные,0.619109,0.968924
1,Сбалансированные,0.691812,0.636854


Точность на тестовой выборке при сбалансированных входных данных существенно меньше,чем на несбалансированных. Это легко объяснить большим перекосом в сторону объектов с ЦД = 0. Даже если б моделирование на всех объъектах расставила нули, она была бы права в юолее, чем 97 процентах случаев. Поэтому стоит ориентироваться на значение ROC-AUC. И по этому показателю сбалансированные данные показывают лучшие результаты. Поэтому далее все методы моделирования будут рассчитываться по сбалансированным данным

### Логистическая регрессия

In [27]:
logreg_bal = LogisticRegression(random_state=42, max_iter=500)

logreg_bal.fit(x_train_bal, y_train_bal)

pred_logreg_bal = logreg_bal.predict(x_test_bal)

In [28]:
print('Результаты моделирования логистической регрессии по сбалансированным данным:')
print('ROC AUC точность модели:', roc_auc_score(y_test_bal, logreg_bal.predict_proba(x_test_bal)[:, 1]))
print(f'Точность на тестовой выборке: {accuracy_score(y_test_bal, pred_logreg_bal)}')
print(f'Матрица ошибок:\n {confusion_matrix(y_test_bal, pred_logreg_bal)}')

Результаты моделирования логистической регрессии по сбалансированным данным:
ROC AUC точность модели: 0.6870879560287578
Точность на тестовой выборке: 0.6295008115538773
Матрица ошибок:
 [[ 8864  6282]
 [ 4903 10140]]


### Многослойный персептрон

In [35]:
mlp_bal = MLPClassifier()

mlp_bal.fit(x_train_bal, y_train_bal)

pred_mlp_bal = mlp_bal.predict(x_test_bal)

C:\Users\shish\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


print('Результаты моделирования многослойного персептрона по сбалансированным данным:')
print('ROC AUC точность модели:', roc_auc_score(y_test_bal, mlp_bal.predict_proba(x_test_bal)[:, 1]))
print(f'Точность на тестовой выборке: {accuracy_score(y_test_bal, pred_mlp_bal)}')
print(f'Матрица ошибок:\n {confusion_matrix(y_test_bal, pred_mlp_bal)}')

## Results

In [48]:
results = {'Тип модели': ['Random_Forest', 'Logistic_Regression', 'MLP'], 
           'ROC_AUC': [roc_auc_score(y_test_bal, rf_clf_bal.predict_proba(x_test_bal)[:, 1]), roc_auc_score(y_test_bal, logreg_bal.predict_proba(x_test_bal)[:, 1]), roc_auc_score(y_test_bal, mlp_bal.predict_proba(x_test_bal)[:, 1])]}
           

In [49]:
df_balance_results = pd.DataFrame(data=results)

In [50]:
df_balance_results

,Тип модели,ROC_AUC
0,Random_Forest,0.691625
1,Logistic_Regression,0.687088
2,MLP,0.660635


Таким образом, наилучший показатель имеет моделирование случайного леса. Изменения параметров моделирования не дало положительных результатов. Также изучил параметр class_weight для моделей случайного леса и логистической регрессии. Протестировал на несбалансированных данных. Результат улучшился, но все равно не превышает текущих значений. К тому же не многослойного перспептрона такая настройка отсутствует.